In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output

Game = Regency()
clear_output()
for a in range(1):
    Game.random_events()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_regency_points()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.domain_initiative()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_gold_bars()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.maintenance_costs()
    print(len((Game.Seasons[Game.Season]['Season'])))

Game.clear_screen()
Game.Seasons[Game.Season]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,MF,False,No Event,59,20,23,4,9,Rich
1,JR,False,No Event,33,20,0,12,12,Dormant
2,Sw2,False,No Event,24,19,1,0,2,Bare
3,HA,False,Feud,3,18,0,3,3,Bare
4,TF,False,Brigandage,2,17,1,2,1,Dormant
5,Ch,False,No Event,28,16,0,0,0,Dormant
6,CoL,False,No Event,27,13,2,15,13,Bare
7,WB,False,No Event,2,12,1,2,1,Dormant
8,LPA,False,No Event,11,12,5,9,4,Bare
9,GG,False,No Event,77,11,54,44,30,Rich


In [2]:
 Game.take_domain_actions(None)

C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])
C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])


C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])


C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])
C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])


Error with provence <source.regency.Regency object at 0x0000019E3E204A20>


C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])


...

Error with provence <source.regency.Regency object at 0x0000019E3E204A20>
Error with provence <source.regency.Regency object at 0x0000019E3E204A20>
Error with provence <source.regency.Regency object at 0x0000019E3E204A20>
Error with provence <source.regency.Regency object at 0x0000019E3E204A20>
Error with provence <source.regency.Regency object at 0x0000019E3E204A20>


C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])
C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])


C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])


C:\git\Regency\source\DQNAgent.py:259: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  rtemp = pd.concat([prov_regent_list, rtemp])


AttributeError: 'Regency' object has no attribute 'clear_output'

In [11]:
import numpy as np
import pandas as pd
int(np.random.randint(1,5,1))
self = Game

In [12]:
Provences = ['Gulfport']
Regent = 'HA'
Target = 'JR'
Conflict = True
bid=None
cost = 1

dc = 10
# spend the money and regency
self.change_regent(Regent, Gold_Bars = self.Regents[self.Regents['Regent']==Regent]['Gold Bars'].values[0] - cost, Regency_Points = self.Regents[self.Regents['Regent']==Regent]['Regency Points'].values[0] - cost)
# set the dc
adj = (Conflict == False)
dc = self.set_difficulty(dc, Regent, Target, hostile=Conflict)
# roll 'em
success, crit = self.make_roll(Regent, dc, 'Persuasion', adj=False, dis=False, player_gbid=bid)
add = -1*Conflict + 1-Conflict
dip = add
if crit:
    add = 2*add
if success:
    # make the changes
    temp = {}
    temp['Provence'] = Provences
    temp = pd.DataFrame(temp)
    temp = pd.merge(temp, self.Provences.copy(), on='Provence', how='left')
    temp['Loyalty'] = temp['Loyalty'].str.replace('Rebellious','0').replace('Poor','1').replace('Average','2').replace('High','3').astype(int) + add 
    temp['Loyalty'] = temp['Loyalty'].astype(str).str.replace('-2', 'Rebellious').replace('-1', 'Rebellious').replace('0', 'Rebellious').replace('1', 'Poor').replace('2', 'Average').replace('3', 'High').replace('4', 'High').replace('5', 'High')
    for i, row in temp.iterrows():
        self.change_provence(self, row['Provence'], Loyalty=row['Loyalty'])

    if dip == -1:
        message = '{} agitated the people against {} in the following provences: {}'.format(Regent, Target, ', '.join(Provences))
    else:
        message = '{} used their influence to rally the people behind {} in the following provences: {}'.format(Regent, Target, ', '.join(Provences))
    # change diplomacy (Traget cares)
    self.add_relationship(Target, Regent, Diplomacy=dip)
    temp = self.Relationships[self.Relationships['Regent']==Regent].copy()
    if temp[temp['Other']==Target]['Diplomacy'].shape[0] > 0:
        reward = dip*temp[temp['Other']==Target]['Diplomacy'].values[0]
    else:
        reward = 0
    attitude = self.Regents[self.Regents['Regent']==Regent]['Attitude']
else:
    message = '{} failed to change peoples minds about {} in {}'.format(Regent, Target, ', '.join(Provences))
    # change diplomacy (Traget cares)
    self.add_relationship(Target, Regent, Diplomacy=dip)
    reward = 0


Error with provence <source.regency.Regency object at 0x0000019E3E204A20>


In [13]:
row['Provence']

'Gulfport'

In [15]:
 self.change_provence(self, 'Gulfport', Loyalty=row['Loyalty'])

Error with provence <source.regency.Regency object at 0x0000019E3E204A20>


In [17]:
self.Provences[self.Provences['Provence'] == row['Provence']]

,Provence,Domain,Region,Regent,Terrain,Loyalty,Taxation,Population,Magic,Castle,Capital,Position,Contested,Waterway,Brigands
10,Gulfport,Osoerde,Eastern Marches,JR,Plains,Average,Moderate,3,2,0,False,"[1515, 2154]",False,True,False


In [25]:
Provence = 'Gulfport'
Regent=None
Region=None
Domain=None
Population_Change=0
Terrain=None
Loyalty='Poop'
Taxation=None
Castle=None
Capital=None
x=None
y=None
Contested=None
Waterway=None
Brigands=False

index = self.Provences.index[self.Provences['Provence'] == Provence].tolist()[0]
old = self.Provences.loc[index]
if Regent == None:
    Regent = old['Regent']
if Domain == None:
    Domain = old['Domain']
if Region == None:
    Region = old['Region']
if Terrain == None:
    Terrain = old['Terrain']
if Contested == None:
    Contested = old['Contested']
if Waterway == None:
    Waterway = old['Waterway']
try:
    loy = old['Loyalty'].replace('Rebellious','0').replace('Poor','1').replace('Average','2').replace('High','3')
    new = int(loy) + int(Loyalty)
    if new <= 0:
        Loyalty = 'Rebellious'
    if new == 1:
        Loyalty = 'Poor'
    if new == 2:
        Loyalty = 'Average'
    if new >= 3:
        Loyalty = 'High'
except:
    if Loyalty == None:
        Loyalty = old['Loyalty']
if Taxation == None:
    Taxation == old['Taxation']

Population = old['Population'] + Population_Change
Magic = old['Magic'] - Population_Change

if Magic <= 0:
    Magic = 1
if Castle == None:
    Castle = old['Castle']
if Capital == None:
    Capital = old['Capital']
if x == None or y == None:
    pos = old['Position']
else:
    pos = np.array([x, y])
self.Provences.loc[index] = [Provence, Domain, Region, Regent, Terrain, Loyalty, Taxation,
    Population, Magic, Castle, Capital, old['Position'], Contested, Waterway, Brigands]

self.Provences.loc[index]

Provence             Gulfport
Domain                Osoerde
Region        Eastern Marches
Regent                     JR
Terrain                Plains
Loyalty                  Poop
Taxation                 None
Population                  3
Magic                       2
Castle                      0
Capital                 False
Position         [1515, 2154]
Contested               False
Waterway                 True
Brigands                False
Name: 10, dtype: object

In [ ]:
Game.change_provence('Gulfport', Loyalty)